# Data analysis of drug activity in St Roch

The drug activity we've witnessed on Music St and N Villere St seems to be more widespread than just this area. This notebook agregates the data we've gathered to understand how this activity is spread throughout the ward.

### Library imports

In [3]:
# Web tools / scraping
import requests
from bs4 import BeautifulSoup

# Geo location
from geopy.geocoders import ArcGIS

# Mapping library
import folium

### Download ownership and mailing address information from the New Orleans Assessor

In [8]:
class Address():
    def __init__(self, number, street, street_type, city="New Orleans", state="LA", zip_code="70117"):
        self.number = number
        self.street = street
        self.street_type = street_type
        self.city = city
        self.state = state
        self.zip_code = zip_code

    @property
    def full_address(self):
        return "{} {} {}, {}, {} {}".format(self.number, self.street, self.street_type, self.city, self.state, self.zip_code)

class Property():
    def __init__(self, owner_name, address, contact_address):
        self.owner_name = owner_name
        self.address = address
        self.contact_address = contact_address


In [100]:
root_url = "http://qpublic9.qpublic.net/la_orleans_display.php"
key_base = "?KEY="
addresses = [
#     "1420-MUSICST",
#     "1338-MUSICST",
    "2421-NVILLEREST",
#     "2418-NVILLEREST",
#     "2323-NROMANST",
#     "1400-MUSICST",
#     "1404-MUSICST"
]

properties = []
for address in addresses:
    url = root_url + key_base + address
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    owner_info = soup.find_all("td", class_='owner_value')
#     print(type(owner_info[0]))
#     print(owner_info[0].contents)
#     print(owner_info[2].contents)
#     print(owner_info[4].contents)

    # Get this property's street address from the record
    # Replace any encoded values with a space, split into a list, and remove any empty elements
    street_address = list( filter( None, owner_info[4].text.replace(u'\xa0', ' ').split(' ') ) )
    number = street_address.pop(0).strip()
    street_type = street_address.pop(-1).strip()
    street = " ".join(street_address).strip()
    address = Address(number, street, street_type)

    # Get this property's contact address from the record
    contact_address = owner_info[2].contents
    print(contact_address)
    linebreak = BeautifulSoup('<br/>')
    contact_address = list( filter( linebreak.br, contact_address ) )    
    print(contact_address)
    namenumberstreet, citystatezip = contact_address.split('\xa0\xa0')

    city, statezip = citystatezip.strip().split(',')
    state, zip_code = statezip.strip().split(' ')

    namenumberstreet = list( filter( None, namenumberstreet.split('\xa0') ) )
    namenumberstreet = [e.split(' ') for e in namenumberstreet]
    flat_list = []
    for e in namenumberstreet:
        flat_list += e
    namenumberstreet = flat_list

    for i, element in enumerate(namenumberstreet):
        print(i , element,element.strip().isnumeric())
        if element.strip().isnumeric():
            number_index = i
            break
    name = namenumberstreet[:number_index]
    numberstreet = namenumberstreet[number_index:]
    print(name, numberstreet)
#     print(namestreet.split('\xa0')[:i], namestreet.split('\xa0')[i:])
#     namestreet = list( filter( None, namestreet.split('\xa0') ) )
    if len(namestreet) > 1:
        name, street = namestreet
    else:
        street = " ".join(namestreet)

    street = street.split(' ')
    number = street.pop(0)
    street_type = street.pop(-1)
    street = " ".join(street)

    contact_address = Address(number, street, street_type, city, state, zip_code)
    print(contact_address.full_address)
    owner_name = owner_info[0].text
    properties.append(Property(owner_name, address, contact_address))

#     print(";".join([owner_info[0].text, owner_info[4].text, owner_info[2].text]))

# If the mailing address does not match the property address, get the information for the mailing address.

['\xa014175 CYPRESS N HOUSTON', <br/>, '\xa0APT. # 606\xa0', <br/>, '\xa0CYPRESS, TX 77429']
[]


AttributeError: 'list' object has no attribute 'split'

### Convert all addresses to latitude-longitude coordinates

In [14]:
locator = ArcGIS()
locations = [locator.geocode(p.address.full_address) for p in properties]
coordinates = [[l.latitude, l.longitude] for l in locations]

[[29.972443861503848, -90.05132342146109], [29.97179929808414, -90.05127266417561], [29.971983280984187, -90.05168298521465], [29.971880686363953, -90.05173286441803], [29.975840779358247, -90.05437738703907], [29.972025000000002, -90.051372], [29.97211010713255, -90.05129789884005]]


### Plot the addresses and their relationships

In [21]:
fmap = folium.Map(location=coordinates[0], zoom_start=17)
for coordinate in coordinates:
    folium.Marker(location=coordinate, icon=folium.Icon(color='green')).add_to(fmap)
# folium.PolyLine(locations=[p1, p2], color='blue').add_to(some_map)
fmap